In [53]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime


In [54]:
# Setup cache and retry settings
cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


In [55]:
# Define parameters for API request with both hourly and daily variables
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 48.8566,  # Paris latitude
    "longitude": 2.3522,  # Paris longitude
    "start_date": "2020-01-01",
    "end_date": datetime.now().strftime("%Y-%m-%d"),
    "hourly": (
        "temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,"
        "surface_pressure,precipitation,rain,snowfall,cloud_cover,"
        "cloud_cover_low,cloud_cover_mid,cloud_cover_high,shortwave_radiation,"
        "direct_radiation,direct_normal_irradiance,diffuse_radiation,wind_speed_10m,"
        "wind_direction_10m,wind_gusts_10m,et0_fao_evapotranspiration,"
        "weather_code,snow_depth,vapour_pressure_deficit,"
        "soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,"
        "soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,"
        "soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,"
        "soil_moisture_28_to_100cm,soil_moisture_100_to_255cm"
    ),
    "daily": (
        "temperature_2m_min,temperature_2m_max,precipitation_sum,"
        "rain_sum,snowfall_sum,precipitation_hours,sunshine_duration,"
        "daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,"
        "shortwave_radiation_sum,et0_fao_evapotranspiration_sum"
    ),
    "timezone": "auto",
}


In [56]:
# Fetch data from API
responses = openmeteo.weather_api(url, params=params)
response = responses[0]  # First response for Paris

# Display metadata
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 48.82249450683594°N 2.288135528564453°E
Elevation 36.0 m asl
Timezone b'Europe/Paris' b'CET'
Timezone difference to GMT+0 3600 s


In [57]:
# Process hourly data
hourly = response.Hourly()
hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    )
}


In [58]:
# Organize all variables into a dictionary for DataFrame creation
hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    ),
    "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
    "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
    "dew_point_2m": hourly.Variables(2).ValuesAsNumpy(),
    "apparent_temperature": hourly.Variables(3).ValuesAsNumpy(),
    "surface_pressure": hourly.Variables(4).ValuesAsNumpy(),
    "precipitation": hourly.Variables(5).ValuesAsNumpy(),
    "rain": hourly.Variables(6).ValuesAsNumpy(),
    "snowfall": hourly.Variables(7).ValuesAsNumpy(),
    "cloud_cover": hourly.Variables(8).ValuesAsNumpy(),
    "cloud_cover_low": hourly.Variables(9).ValuesAsNumpy(),
    "cloud_cover_mid": hourly.Variables(10).ValuesAsNumpy(),
    "cloud_cover_high": hourly.Variables(11).ValuesAsNumpy(),
    "shortwave_radiation": hourly.Variables(12).ValuesAsNumpy(),
    "direct_radiation": hourly.Variables(13).ValuesAsNumpy(),
    "direct_normal_irradiance": hourly.Variables(14).ValuesAsNumpy(),
    "diffuse_radiation": hourly.Variables(15).ValuesAsNumpy(),
    "wind_speed_10m": hourly.Variables(16).ValuesAsNumpy(),
    "wind_direction_10m": hourly.Variables(17).ValuesAsNumpy(),
    "wind_gusts_10m": hourly.Variables(18).ValuesAsNumpy(),
    "et0_fao_evapotranspiration": hourly.Variables(19).ValuesAsNumpy(),
    "weather_code": hourly.Variables(20).ValuesAsNumpy(),
    "snow_depth": hourly.Variables(21).ValuesAsNumpy(),
    "vapour_pressure_deficit": hourly.Variables(22).ValuesAsNumpy(),
    "soil_temperature_0_to_7cm": hourly.Variables(23).ValuesAsNumpy(),
    "soil_temperature_7_to_28cm": hourly.Variables(24).ValuesAsNumpy(),
    "soil_temperature_28_to_100cm": hourly.Variables(25).ValuesAsNumpy(),
    "soil_temperature_100_to_255cm": hourly.Variables(26).ValuesAsNumpy(),
    "soil_moisture_0_to_7cm": hourly.Variables(27).ValuesAsNumpy(),
    "soil_moisture_7_to_28cm": hourly.Variables(28).ValuesAsNumpy(),
    "soil_moisture_28_to_100cm": hourly.Variables(29).ValuesAsNumpy(),
    "soil_moisture_100_to_255cm": hourly.Variables(30).ValuesAsNumpy(),
}

# Create a DataFrame
hourly_dataframe = pd.DataFrame(data=hourly_data)
print(hourly_dataframe)


                           date  temperature_2m  relative_humidity_2m  \
0     2019-12-31 23:00:00+00:00           -0.14             99.276306   
1     2020-01-01 00:00:00+00:00            2.61             98.239220   
2     2020-01-01 01:00:00+00:00            2.11             99.644173   
3     2020-01-01 02:00:00+00:00            1.91            100.000000   
4     2020-01-01 03:00:00+00:00            2.21             97.883568   
...                         ...             ...                   ...   
42355 2024-10-30 18:00:00+00:00             NaN                   NaN   
42356 2024-10-30 19:00:00+00:00             NaN                   NaN   
42357 2024-10-30 20:00:00+00:00             NaN                   NaN   
42358 2024-10-30 21:00:00+00:00             NaN                   NaN   
42359 2024-10-30 22:00:00+00:00             NaN                   NaN   

       dew_point_2m  apparent_temperature  surface_pressure  precipitation  \
0             -0.24             -2.673424    

In [59]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42360 entries, 0 to 42359
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   date                           42360 non-null  datetime64[ns, UTC]
 1   temperature_2m                 42314 non-null  float32            
 2   relative_humidity_2m           42314 non-null  float32            
 3   dew_point_2m                   42314 non-null  float32            
 4   apparent_temperature           42314 non-null  float32            
 5   surface_pressure               42314 non-null  float32            
 6   precipitation                  42314 non-null  float32            
 7   rain                           42314 non-null  float32            
 8   snowfall                       42314 non-null  float32            
 9   cloud_cover                    42314 non-null  float32            
 10  cloud_cover_low       

In [60]:
# Which dates have null values?
hourly_dataframe[hourly_dataframe.isnull().any(axis=1)]["date"]

39841   2024-07-18 00:00:00+00:00
39842   2024-07-18 01:00:00+00:00
39843   2024-07-18 02:00:00+00:00
39844   2024-07-18 03:00:00+00:00
39845   2024-07-18 04:00:00+00:00
                   ...           
42355   2024-10-30 18:00:00+00:00
42356   2024-10-30 19:00:00+00:00
42357   2024-10-30 20:00:00+00:00
42358   2024-10-30 21:00:00+00:00
42359   2024-10-30 22:00:00+00:00
Name: date, Length: 623, dtype: datetime64[ns, UTC]

In [61]:
# Process daily data
daily = response.Daily()
daily_data_length = len(daily.Variables(0).ValuesAsNumpy())
dates_daily = pd.date_range(
    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
    periods=daily_data_length,
    freq="D",
)

# Define daily data dictionary
daily_data = {
    "date": dates_daily,
    "temperature_2m_min": daily.Variables(0).ValuesAsNumpy(),
    "temperature_2m_max": daily.Variables(1).ValuesAsNumpy(),
    "precipitation_sum": daily.Variables(2).ValuesAsNumpy(),
    "rain_sum": daily.Variables(3).ValuesAsNumpy(),
    "snowfall_sum": daily.Variables(4).ValuesAsNumpy(),
    "precipitation_hours": daily.Variables(5).ValuesAsNumpy(),
    "sunshine_duration": daily.Variables(6).ValuesAsNumpy(),
    "daylight_duration": daily.Variables(7).ValuesAsNumpy(),
    "wind_speed_10m_max": daily.Variables(8).ValuesAsNumpy(),
    "wind_gusts_10m_max": daily.Variables(9).ValuesAsNumpy(),
    "shortwave_radiation_sum": daily.Variables(10).ValuesAsNumpy(),
    "et0_fao_evapotranspiration_sum": daily.Variables(11).ValuesAsNumpy(),
}


In [62]:
# Create daily DataFrame
daily_dataframe = pd.DataFrame(data=daily_data)

# Check for missing values summary
print("Hourly Data - Missing values per column:\n", hourly_dataframe.isnull().sum())
print("Daily Data - Missing values per column:\n", daily_dataframe.isnull().sum())


Hourly Data - Missing values per column:
 date                               0
temperature_2m                    46
relative_humidity_2m              46
dew_point_2m                      46
apparent_temperature              46
surface_pressure                  46
precipitation                     46
rain                              46
snowfall                          46
cloud_cover                       46
cloud_cover_low                   46
cloud_cover_mid                   46
cloud_cover_high                  46
shortwave_radiation               46
direct_radiation                  46
direct_normal_irradiance          46
diffuse_radiation                 46
wind_speed_10m                    46
wind_direction_10m                46
wind_gusts_10m                    46
et0_fao_evapotranspiration        46
weather_code                      46
snow_depth                       623
vapour_pressure_deficit           46
soil_temperature_0_to_7cm         46
soil_temperature_7_to_28cm       

In [63]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1765 entries, 0 to 1764
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   date                            1765 non-null   datetime64[ns, UTC]
 1   temperature_2m_min              1764 non-null   float32            
 2   temperature_2m_max              1764 non-null   float32            
 3   precipitation_sum               1763 non-null   float32            
 4   rain_sum                        1763 non-null   float32            
 5   snowfall_sum                    1763 non-null   float32            
 6   precipitation_hours             1765 non-null   float32            
 7   sunshine_duration               1763 non-null   float32            
 8   daylight_duration               1765 non-null   float32            
 9   wind_speed_10m_max              1764 non-null   float32            
 10  wind_gusts_1

In [64]:
df_hourly_missing_values = pd.read_csv("../data/open-meteo-48.82N2.29E43m.csv")

In [65]:
df_hourly_missing_values.fillna(value=0, inplace=True)

In [66]:
df_hourly_missing_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   time                                504 non-null    object 
 1   temperature_2m (°C)                 504 non-null    float64
 2   relative_humidity_2m (%)            504 non-null    int64  
 3   dew_point_2m (°C)                   504 non-null    float64
 4   apparent_temperature (°C)           504 non-null    float64
 5   precipitation (mm)                  504 non-null    float64
 6   rain (mm)                           504 non-null    float64
 7   snowfall (cm)                       504 non-null    float64
 8   snow_depth (m)                      504 non-null    float64
 9   weather_code (wmo code)             504 non-null    int64  
 10  pressure_msl (hPa)                  504 non-null    float64
 11  surface_pressure (hPa)              504 non-n

In [67]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42360 entries, 0 to 42359
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   date                           42360 non-null  datetime64[ns, UTC]
 1   temperature_2m                 42314 non-null  float32            
 2   relative_humidity_2m           42314 non-null  float32            
 3   dew_point_2m                   42314 non-null  float32            
 4   apparent_temperature           42314 non-null  float32            
 5   surface_pressure               42314 non-null  float32            
 6   precipitation                  42314 non-null  float32            
 7   rain                           42314 non-null  float32            
 8   snowfall                       42314 non-null  float32            
 9   cloud_cover                    42314 non-null  float32            
 10  cloud_cover_low       

In [70]:
# Step 1: Align column names and format the date
df_hourly_missing_values = df_hourly_missing_values.rename(
    columns={
        "time": "date",  # Rename time column to date to match hourly_dataframe
        "temperature_2m (°C)": "temperature_2m",
        "relative_humidity_2m (%)": "relative_humidity_2m",
        "dew_point_2m (°C)": "dew_point_2m",
        "apparent_temperature (°C)": "apparent_temperature",
        "precipitation (mm)": "precipitation",
        "rain (mm)": "rain",
        "snowfall (cm)": "snowfall",
        "snow_depth (m)": "snow_depth",
        "weather_code (wmo code)": "weather_code",
        "pressure_msl (hPa)": "pressure_msl",
        "surface_pressure (hPa)": "surface_pressure",
        "cloud_cover (%)": "cloud_cover",
        "cloud_cover_low (%)": "cloud_cover_low",
        "cloud_cover_mid (%)": "cloud_cover_mid",
        "cloud_cover_high (%)": "cloud_cover_high",
        "et0_fao_evapotranspiration (mm)": "et0_fao_evapotranspiration",
        "vapour_pressure_deficit (kPa)": "vapour_pressure_deficit",
        "wind_speed_10m (km/h)": "wind_speed_10m",
        "wind_speed_100m (km/h)": "wind_speed_100m",
        "wind_direction_10m (°)": "wind_direction_10m",
        "wind_direction_100m (°)": "wind_direction_100m",
        "wind_gusts_10m (km/h)": "wind_gusts_10m",
        "soil_temperature_0_to_7cm (°C)": "soil_temperature_0_to_7cm",
        "soil_temperature_7_to_28cm (°C)": "soil_temperature_7_to_28cm",
        "soil_temperature_28_to_100cm (°C)": "soil_temperature_28_to_100cm",
        "soil_temperature_100_to_255cm (°C)": "soil_temperature_100_to_255cm",
        "soil_moisture_0_to_7cm (m³/m³)": "soil_moisture_0_to_7cm",
        "soil_moisture_7_to_28cm (m³/m³)": "soil_moisture_7_to_28cm",
        "soil_moisture_28_to_100cm (m³/m³)": "soil_moisture_28_to_100cm",
        "soil_moisture_100_to_255cm (m³/m³)": "soil_moisture_100_to_255cm",
    }
)

# Convert date columns to datetime to ensure matching format
df_hourly_missing_values["date"] = pd.to_datetime(
    df_hourly_missing_values["date"]
).dt.tz_localize("CET", ambiguous="NaT", nonexistent="NaT")

# Step 2: Merge dataframes
merged_df = (
    hourly_dataframe.set_index("date")
    .combine_first(df_hourly_missing_values.set_index("date"))
    .reset_index()
)

# Step 3: Verify the result
merged_df.info()  # Check if the missing values are filled


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42360 entries, 0 to 42359
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   date                           42360 non-null  datetime64[ns, UTC]
 1   apparent_temperature           42314 non-null  float64            
 2   cloud_cover                    42314 non-null  float64            
 3   cloud_cover_high               42314 non-null  float64            
 4   cloud_cover_low                42314 non-null  float64            
 5   cloud_cover_mid                42314 non-null  float64            
 6   dew_point_2m                   42314 non-null  float64            
 7   diffuse_radiation              42314 non-null  float32            
 8   direct_normal_irradiance       42314 non-null  float32            
 9   direct_radiation               42314 non-null  float32            
 10  et0_fao_evapotranspira

In [71]:
merged_df.drop(
    columns=["pressure_msl", "wind_direction_100m", "wind_speed_100m"], inplace=True
)

In [72]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42360 entries, 0 to 42359
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   date                           42360 non-null  datetime64[ns, UTC]
 1   apparent_temperature           42314 non-null  float64            
 2   cloud_cover                    42314 non-null  float64            
 3   cloud_cover_high               42314 non-null  float64            
 4   cloud_cover_low                42314 non-null  float64            
 5   cloud_cover_mid                42314 non-null  float64            
 6   dew_point_2m                   42314 non-null  float64            
 7   diffuse_radiation              42314 non-null  float32            
 8   direct_normal_irradiance       42314 non-null  float32            
 9   direct_radiation               42314 non-null  float32            
 10  et0_fao_evapotranspira

In [73]:
hourly_dataframe_corrected = merged_df.copy()

In [74]:
hourly_dataframe_corrected

,date,apparent_temperature,cloud_cover,cloud_cover_high,cloud_cover_low,cloud_cover_mid,dew_point_2m,diffuse_radiation,direct_normal_irradiance,direct_radiation,...,soil_temperature_100_to_255cm,soil_temperature_28_to_100cm,soil_temperature_7_to_28cm,surface_pressure,temperature_2m,vapour_pressure_deficit,weather_code,wind_direction_10m,wind_gusts_10m,wind_speed_10m
0,2019-12-31 23:00:00+00:00,-2.673424,90.000000,0.0,100.0,0.0,-0.24,0.0,0.0,0.0,...,12.01,8.26,3.36,1028.159790,-0.14,0.004391,3.0,81.869987,5.760000,2.545584
1,2020-01-01 00:00:00+00:00,0.337214,87.299995,0.0,97.0,0.0,2.36,0.0,0.0,0.0,...,12.01,8.26,4.11,1027.808105,2.61,0.013017,3.0,119.054512,7.559999,3.706427
2,2020-01-01 01:00:00+00:00,-0.233696,85.500000,0.0,95.0,0.0,2.06,0.0,0.0,0.0,...,12.01,8.21,4.06,1027.500977,2.11,0.002539,3.0,131.185822,7.200000,3.826853
3,2020-01-01 02:00:00+00:00,-0.615977,84.599998,0.0,94.0,0.0,1.91,0.0,0.0,0.0,...,12.01,8.21,4.01,1027.397827,1.91,0.000000,3.0,162.897186,8.640000,4.896529
4,2020-01-01 03:00:00+00:00,-0.663568,84.599998,0.0,94.0,0.0,1.91,0.0,0.0,0.0,...,12.01,8.21,4.01,1026.905273,2.21,0.015208,3.0,171.469315,13.320000,7.280550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42355,2024-10-30 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42356,2024-10-30 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42357,2024-10-30 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42358,2024-10-30 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# Print dates with missing values
hourly_dataframe_corrected[hourly_dataframe_corrected.isnull().any(axis=1)]["date"]

42217   2024-10-25 00:00:00+00:00
42218   2024-10-25 01:00:00+00:00
42219   2024-10-25 02:00:00+00:00
42220   2024-10-25 03:00:00+00:00
42221   2024-10-25 04:00:00+00:00
                   ...           
42355   2024-10-30 18:00:00+00:00
42356   2024-10-30 19:00:00+00:00
42357   2024-10-30 20:00:00+00:00
42358   2024-10-30 21:00:00+00:00
42359   2024-10-30 22:00:00+00:00
Name: date, Length: 143, dtype: datetime64[ns, UTC]

In [76]:
df_daily_missing_values = pd.read_csv(
    "../data/open-meteo-48.82N2.29E43m-daily-variables.csv"
)


In [77]:
df_daily_missing_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   time                             21 non-null     object 
 1   weather_code (wmo code)          21 non-null     int64  
 2   temperature_2m_max (°C)          21 non-null     float64
 3   temperature_2m_min (°C)          21 non-null     float64
 4   daylight_duration (s)            21 non-null     float64
 5   sunshine_duration (s)            21 non-null     float64
 6   precipitation_sum (mm)           21 non-null     float64
 7   rain_sum (mm)                    21 non-null     float64
 8   snowfall_sum (cm)                21 non-null     float64
 9   precipitation_hours (h)          21 non-null     float64
 10  wind_speed_10m_max (km/h)        21 non-null     float64
 11  wind_gusts_10m_max (km/h)        21 non-null     float64
 12  shortwave_radiation_sum 

In [80]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1765 entries, 0 to 1764
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   date                            1765 non-null   datetime64[ns, UTC]
 1   temperature_2m_min              1764 non-null   float32            
 2   temperature_2m_max              1764 non-null   float32            
 3   precipitation_sum               1763 non-null   float32            
 4   rain_sum                        1763 non-null   float32            
 5   snowfall_sum                    1763 non-null   float32            
 6   precipitation_hours             1765 non-null   float32            
 7   sunshine_duration               1763 non-null   float32            
 8   daylight_duration               1765 non-null   float32            
 9   wind_speed_10m_max              1764 non-null   float32            
 10  wind_gusts_1

In [82]:
df_daily_missing_values

,date,weather_code (wmo code),temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration_sum
0,2024-07-18,2,29.0,14.7,56343.93,50569.87,0.0,0.0,0.0,0.0,8.5,17.6,25.70,5.16
1,2024-07-19,51,31.8,17.8,56212.32,48722.64,0.2,0.2,0.0,2.0,8.4,16.6,23.37,4.97
2,2024-07-20,63,30.2,18.7,56077.21,50603.54,13.6,13.6,0.0,6.0,12.7,28.4,23.72,5.20
3,2024-07-21,51,25.3,18.3,55938.77,35298.14,1.6,1.6,0.0,13.0,19.7,37.8,17.27,3.65
4,2024-07-22,3,24.5,14.5,55797.21,29176.63,0.0,0.0,0.0,0.0,17.4,33.1,15.83,3.69
5,2024-07-23,53,23.7,16.9,55652.70,33998.34,1.9,1.9,0.0,8.0,24.5,46.4,17.68,3.81
6,2024-07-24,3,24.4,13.8,55505.41,43260.05,0.0,0.0,0.0,0.0,7.0,16.2,19.06,3.56
7,2024-07-25,51,27.9,15.6,55355.52,39610.03,0.3,0.3,0.0,1.0,13.3,28.1,21.14,4.82
8,2024-07-26,53,23.6,19.1,55203.22,11893.46,5.0,5.0,0.0,14.0,10.6,21.6,11.44,2.33
9,2024-07-27,61,19.7,16.1,55048.66,648.79,7.6,7.6,0.0,17.0,10.2,21.2,6.66,1.35


In [83]:
# Step 1: Align column names
df_daily_missing_values = df_daily_missing_values.rename(
    columns={
        "time": "date",
        "temperature_2m_max (°C)": "temperature_2m_max",
        "temperature_2m_min (°C)": "temperature_2m_min",
        "daylight_duration (s)": "daylight_duration",
        "sunshine_duration (s)": "sunshine_duration",
        "precipitation_sum (mm)": "precipitation_sum",
        "rain_sum (mm)": "rain_sum",
        "snowfall_sum (cm)": "snowfall_sum",
        "precipitation_hours (h)": "precipitation_hours",
        "wind_speed_10m_max (km/h)": "wind_speed_10m_max",
        "wind_gusts_10m_max (km/h)": "wind_gusts_10m_max",
        "shortwave_radiation_sum (MJ/m²)": "shortwave_radiation_sum",
        "et0_fao_evapotranspiration (mm)": "et0_fao_evapotranspiration_sum",
    }
)

# Ensure 'date' is parsed as datetime before applying timezone localization
df_daily_missing_values["date"] = pd.to_datetime(df_daily_missing_values["date"])

# Now, apply CET timezone localization
df_daily_missing_values["date"] = df_daily_missing_values["date"].dt.tz_localize(
    "CET", ambiguous="NaT", nonexistent="NaT"
)

# Step 2: Merge dataframes and fill missing values
merged_daily_df = (
    daily_dataframe.set_index("date")
    .combine_first(df_daily_missing_values.set_index("date"))
    .reset_index()
)

# Step 3: Verify the merged data
merged_daily_df.info()  # This will show if missing values are filled


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1786 entries, 0 to 1785
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   date                            1786 non-null   datetime64[ns, UTC]
 1   daylight_duration               1786 non-null   float64            
 2   et0_fao_evapotranspiration_sum  1784 non-null   float64            
 3   precipitation_hours             1786 non-null   float64            
 4   precipitation_sum               1784 non-null   float64            
 5   rain_sum                        1784 non-null   float64            
 6   shortwave_radiation_sum         1784 non-null   float64            
 7   snowfall_sum                    1784 non-null   float64            
 8   sunshine_duration               1784 non-null   float64            
 9   temperature_2m_max              1785 non-null   float64            
 10  temperature_

In [84]:
merged_daily_df.drop(columns=["weather_code (wmo code)"], inplace=True)

In [85]:
daily_dataframe_corrected = merged_daily_df.copy()

In [86]:
daily_dataframe_corrected

,date,daylight_duration,et0_fao_evapotranspiration_sum,precipitation_hours,precipitation_sum,rain_sum,shortwave_radiation_sum,snowfall_sum,sunshine_duration,temperature_2m_max,temperature_2m_min,wind_gusts_10m_max,wind_speed_10m_max
0,2019-12-31 23:00:00+00:00,30005.021484,0.196938,5.0,0.500000,0.500000,1.58,0.0,0.000000,5.660000,-0.140000,23.759998,11.681987
1,2020-01-01 23:00:00+00:00,30067.091797,0.303851,9.0,1.500000,1.500000,2.16,0.0,10429.040039,9.310000,5.210000,36.360001,19.855558
2,2020-01-02 23:00:00+00:00,30134.396484,0.544292,7.0,1.000000,1.000000,2.22,0.0,11011.313477,11.510000,8.160000,46.799999,25.161400
3,2020-01-03 23:00:00+00:00,30206.789062,0.538588,0.0,0.000000,0.000000,4.28,0.0,21072.218750,8.710000,3.110000,32.760002,16.119801
4,2020-01-04 23:00:00+00:00,30284.138672,0.420078,0.0,0.000000,0.000000,3.75,0.0,24806.644531,8.960000,2.710000,15.840000,7.289444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,2024-10-25 23:00:00+00:00,36687.621094,0.871624,12.0,4.699999,4.699999,5.72,0.0,17721.390625,18.160000,12.310000,20.519999,6.877789
1782,2024-10-26 23:00:00+00:00,36486.011719,0.609350,5.0,2.800000,2.800000,3.18,0.0,1329.731445,16.360001,13.110001,20.519999,10.209348
1783,2024-10-27 23:00:00+00:00,36285.320312,0.914235,0.0,0.000000,0.000000,5.57,0.0,14961.370117,17.610001,10.360001,19.440001,9.668195
1784,2024-10-28 23:00:00+00:00,36085.750000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,10.210000,9.460000,11.520000,7.029225


In [87]:
# Dates with missing values in daily dataframe corrected
daily_dataframe_corrected[daily_dataframe_corrected.isnull().any(axis=1)]["date"]


1784   2024-10-28 23:00:00+00:00
1785   2024-10-29 23:00:00+00:00
Name: date, dtype: datetime64[ns, UTC]

In [89]:
# check duplicates
daily_dataframe_corrected.duplicated().sum()

np.int64(0)

In [90]:
# Ensure `date` column in `daily_dataframe` is of datetime type and aligns at midnight
daily_dataframe_corrected["date"] = pd.to_datetime(
    daily_dataframe_corrected["date"]
).dt.floor("D")


In [91]:
daily_dataframe_corrected

,date,daylight_duration,et0_fao_evapotranspiration_sum,precipitation_hours,precipitation_sum,rain_sum,shortwave_radiation_sum,snowfall_sum,sunshine_duration,temperature_2m_max,temperature_2m_min,wind_gusts_10m_max,wind_speed_10m_max
0,2019-12-31 00:00:00+00:00,30005.021484,0.196938,5.0,0.500000,0.500000,1.58,0.0,0.000000,5.660000,-0.140000,23.759998,11.681987
1,2020-01-01 00:00:00+00:00,30067.091797,0.303851,9.0,1.500000,1.500000,2.16,0.0,10429.040039,9.310000,5.210000,36.360001,19.855558
2,2020-01-02 00:00:00+00:00,30134.396484,0.544292,7.0,1.000000,1.000000,2.22,0.0,11011.313477,11.510000,8.160000,46.799999,25.161400
3,2020-01-03 00:00:00+00:00,30206.789062,0.538588,0.0,0.000000,0.000000,4.28,0.0,21072.218750,8.710000,3.110000,32.760002,16.119801
4,2020-01-04 00:00:00+00:00,30284.138672,0.420078,0.0,0.000000,0.000000,3.75,0.0,24806.644531,8.960000,2.710000,15.840000,7.289444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,2024-10-25 00:00:00+00:00,36687.621094,0.871624,12.0,4.699999,4.699999,5.72,0.0,17721.390625,18.160000,12.310000,20.519999,6.877789
1782,2024-10-26 00:00:00+00:00,36486.011719,0.609350,5.0,2.800000,2.800000,3.18,0.0,1329.731445,16.360001,13.110001,20.519999,10.209348
1783,2024-10-27 00:00:00+00:00,36285.320312,0.914235,0.0,0.000000,0.000000,5.57,0.0,14961.370117,17.610001,10.360001,19.440001,9.668195
1784,2024-10-28 00:00:00+00:00,36085.750000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,10.210000,9.460000,11.520000,7.029225


In [93]:
# Identify duplicates in the floored 'date' column
duplicate_dates = daily_dataframe_corrected[
    daily_dataframe_corrected.duplicated(subset="date", keep=False)
]
print("Duplicate dates after flooring:\n", duplicate_dates)


Duplicate dates after flooring:
                           date  daylight_duration  \
1660 2024-07-17 00:00:00+00:00       56343.930000   
1661 2024-07-17 00:00:00+00:00       56343.925781   
1662 2024-07-18 00:00:00+00:00       56212.320000   
1663 2024-07-18 00:00:00+00:00       56212.320312   
1664 2024-07-19 00:00:00+00:00       56077.210000   
1665 2024-07-19 00:00:00+00:00       56077.207031   
1666 2024-07-20 00:00:00+00:00       55938.770000   
1667 2024-07-20 00:00:00+00:00       55938.773438   
1668 2024-07-21 00:00:00+00:00       55797.210000   
1669 2024-07-21 00:00:00+00:00       55797.207031   
1670 2024-07-22 00:00:00+00:00       55652.700000   
1671 2024-07-22 00:00:00+00:00       55652.695312   
1672 2024-07-23 00:00:00+00:00       55505.410000   
1673 2024-07-23 00:00:00+00:00       55505.406250   
1674 2024-07-24 00:00:00+00:00       55355.520000   
1675 2024-07-24 00:00:00+00:00       55355.523438   
1676 2024-07-25 00:00:00+00:00       55203.220000   
1677 2024-07-

In [94]:
# Aggregate duplicates by taking the mean of each duplicate group
daily_dataframe_corrected = daily_dataframe_corrected.groupby(
    "date", as_index=False
).mean()

# Check if duplicates are removed
daily_dataframe_corrected.duplicated().sum()

np.int64(0)

In [95]:
# Resample the daily data to hourly by repeating each daily value across all 24 hours
# for each day, then join with the hourly data
daily_expanded = (
    daily_dataframe_corrected.set_index("date").resample("H").ffill().reset_index()
)


C:\Users\ameya\AppData\Local\Temp\ipykernel_20540\1068732823.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_dataframe_corrected.set_index("date").resample("H").ffill().reset_index()


In [98]:
# Merge the expanded daily data with hourly data on the `date` column
merged_dataframe = pd.merge(
    hourly_dataframe_corrected, daily_expanded, on="date", how="left"
)


In [99]:
merged_dataframe.dropna(inplace=True)

In [100]:
merged_dataframe["date"].min(), merged_dataframe["date"].max()

(Timestamp('2019-12-31 23:00:00+0000', tz='UTC'),
 Timestamp('2024-10-24 23:00:00+0000', tz='UTC'))

In [101]:
merged_dataframe.to_csv("../data/paris_weather_extensive.csv", index=False)

In [102]:
merged_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42217 entries, 0 to 42216
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   date                            42217 non-null  datetime64[ns, UTC]
 1   apparent_temperature            42217 non-null  float64            
 2   cloud_cover                     42217 non-null  float64            
 3   cloud_cover_high                42217 non-null  float64            
 4   cloud_cover_low                 42217 non-null  float64            
 5   cloud_cover_mid                 42217 non-null  float64            
 6   dew_point_2m                    42217 non-null  float64            
 7   diffuse_radiation               42217 non-null  float32            
 8   direct_normal_irradiance        42217 non-null  float32            
 9   direct_radiation                42217 non-null  float32            
 10  et0_fao_evapotr